## Natural Language Processing
NLP is a machine learning field that understand the words in a sentence individually and the context behind using those words. Some of the common NLP tasks are - reviewing and classifying whole sentences, identifying each word as part of speech(noun, verb , adjective, entity), generating text with masked words, extracting information from the context, transalting or summarizing the text. 

### Transformers 
One of the mdoels used for solving NLP tasks is the transformers. I will be using the transformers library from Huggingface.
The pipeline API function will connect the pre-trained model with pre-processed inputs and post-prpcessing predictions to directly output the sentiment label of the sentence with accuracy score. For example- Zero-shot classification pipeline allows in annotating the text and classify the sentence into sentimental labels. Text generation pipeline will auto-complete the sentence by generatinhg the remaining text with a provided prompt. Some of the other examples include - Named entity recognition, question answering, summarization, transalation

### Categories of pre-trained transformers models
- GPT-like (auto-regressive)
- BERT-like (auto-encoding)
- BART/T5-like (sequence-to-sequence)

### Transformer architecture
-	Encoder: Encoder encode the input into numerical representation of each word.
-	Decoder: Decoder uses the encoder’s representation along with other inputs to generate a target sequence 
-	Encoder-only models: Bi-directional, for sentiment analysis (sentence classification) and named entity recognition. Ex- BERT, RoBERTa, ALBERT
-	Decoder-only models: unidirectional, for text generation, Ex – CTRL, GPT, GPT-2, Transformer XL
-	Encoder-Decoder or sequence-to-sequence models: translation or summarization, Ex – BART, mBART, Marian, T5

## Using Transformers
Stage 1 (tokenizer)  Stage 2 (Model)  Stage 3 (post processing)

Raw text -->     Input ID     -->       Logits (not probabilities)    -->    Predictions
Tokenizer: 
-	split the input into words, subwords, symbols referred as tokens
-	Mapping each token to an integer
-	“AutoTokenizer” class and “from_pretrained” method


In [68]:
import pandas as pd
# load dataset
df = pd.read_csv(r"C:\Users\minnu\Desktop\Sentiment-analysis-using-NLP\dataset_exploration\all-data.csv", names = ['label', 'sentences'], sep=',', encoding='latin-1')
df.head()

,label,sentences
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [69]:
df_2 = df.copy()

In [70]:
# Preprocessing with Tokenizer
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.9.2",
  "vocab_size": 

In [71]:
raw_inputs = [df_2['sentences'][2],df_2['sentences'][4]] 
inputs = tokenizer(raw_inputs, padding= True, truncation = True, return_tensors = 'pt')
print(inputs)

{'input_ids': tensor([[  101,  1996,  2248,  4816,  3068,  2194,  3449, 19800,  4160,  2038,
          4201,  2125, 15295,  1997,  5126,  2013,  2049, 21169,  4322,  1025,
         10043,  2000,  3041,  3913, 27475,  1996,  2194, 11016,  1996,  6938,
          1997,  2049,  2436,  3667,  1010,  1996,  3679,  2695, 14428,  2229,
          2988,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  2429,  2000,  1996,  2194,  1005,  1055,  7172,  5656,  2005,
          1996,  2086,  2268,  1011,  2262,  1010, 19021,  8059,  7889,  1037,
          2146,  1011,  2744,  5658,  4341,  3930,  1999,  1996,  2846,  1997,
          2322,  1003,  1011,  2871,  1003,  2007,  2019,  4082,  5618,  7785,
          1997,  2184,  1003,  1011,  2322,  1003,  1997,  5658,  4341,  1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [72]:
# Going through the model
from transformers import AutoModel   # AutoModel instantiate the model from checkpoint
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)


loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.9.2",
  "vocab_size": 

In [73]:
# output displaying the batch size, sequence length, hidden vector dimension
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 51, 768])


In [74]:
# Model heads: making sense out of numbers
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs.logits)

loading configuration file https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\4e60bb8efad3d4b7dc9969bf204947c185166a0a3cf37ddb6f481a876a3777b5.9f8326d0b7697c7fd57366cdde57032f46bc10e37ae81cb7eb564d66d23ec96b
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.9.2",
  "vocab_size": 

tensor([[ 1.7797, -1.6082],
        [-0.9567,  0.8728]], grad_fn=<AddmmBackward>)


In [75]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim =-1)
print(predictions)

tensor([[0.9673, 0.0327],
        [0.1383, 0.8617]], grad_fn=<SoftmaxBackward>)


## From tokenizer to model
1. Intialize the BERT model 

In [76]:
# The AutoConfig API allows to instantiate the configuration 
# of a pretrained model from any checkpoint. It contains all the information needed 
# to load the model
'''
from transformers import BertConfig, BertModel 
config = BertConfig()
model = BertModel(config)   # Model is randomly intialized
'''
from transformers import BertModel 
model = BertModel.from_pretrained("bert-base-cased")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/main/pytorch_model.bin from cache at C:\Users\minnu/.cache\h

2. **Tokenizers**: Tokenizers translate raw text inputs into numerical data that can be processed by the model. It could be either word-based, character-based or subword tokenization

In [77]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = df_2["sentences"][2]
tokens = tokenizer.tokenize(sequence)

print(tokens)


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at C:\Users\minnu/.cache\huggingface\trans

['The', 'international', 'electronic', 'industry', 'company', 'El', '##cote', '##q', 'has', 'laid', 'off', 'tens', 'of', 'employees', 'from', 'its', 'Tallinn', 'facility', ';', 'contrary', 'to', 'earlier', 'lay', '##offs', 'the', 'company', 'contracted', 'the', 'ranks', 'of', 'its', 'office', 'workers', ',', 'the', 'daily', 'Post', '##ime', '##es', 'reported', '.']


In [78]:
# From tokens to input IDs
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[1109, 1835, 4828, 2380, 1419, 2896, 21596, 4426, 1144, 3390, 1228, 17265, 1104, 4570, 1121, 1157, 23277, 3695, 132, 11565, 1106, 2206, 3191, 18438, 1103, 1419, 11058, 1103, 6496, 1104, 1157, 1701, 3239, 117, 1103, 3828, 3799, 10453, 1279, 2103, 119]


In [79]:
# Decode
decode_string = tokenizer.decode(ids)
print(decode_string)

The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers, the daily Postimees reported.


3. **Handling multiple sequences**: 
Model expects a batch of inputs, i.e sending multiple sentences through the model all at once. Padding makes sure all the sentences have the same length by adding a special word called padding token to the sentences with fewer values.  To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask. We can truncate the lengths of the sequences by specifying max_sequence_length parameter. 

**After understanding how tokenizer and pretrained model works on few sequences, now it's time to fine-tune the pretrained model on the whole dataset.** 

## Fine-tuning a pretrained model 


The **Trainer** class in transformers help to fine-tune any pretrained models on the dataset. Before defining the Trainer class, we start with dataprocessing. The dataset **financial_phrasebank** needs to be split into train/validation/test dataset before we define the training model by passing all the arguments.  

In [80]:
# Splitting the Data Set into a Training Set and a Test Set
from sklearn.model_selection import train_test_split
sentences = df_2['sentences'].values
target = df_2['label'].values
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, target, test_size=0.2, random_state=1000)

In [81]:
# define the model
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\minnu/.cache\huggingface\t

### Training

In [82]:
# define TrainingArguments class containing all the hyperparameters 
# the Trainer will use for training and evaluation
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [83]:
## Define the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\minnu/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\minnu/.cac

In [84]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = train_texts, 
    eval_dataset = val_texts,
    data_collator=data_collator,
    tokenizer = tokenizer
)


In [85]:
trainer.train()

***** Running training *****
  Num examples = 3876
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1455


AttributeError: 'list' object has no attribute 'keys'